<a href="https://colab.research.google.com/github/josephasal/cosmo_inference/blob/3-parameters/3D_mcmc/3Dmcmc_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file has all the code for the functions:

* Distance modulus + fitting function
* log likelihood and prior
* basic & adaptive mcmc
* Convergence via Gellman Rubin diagnostic
* Autocorrelation
* Effective sample size

except that it is now in 4 dimensions to allow for $\Omega_{\Lambda}$ to be inferred as well $\Omega_k$ to be calculauted at each step

Need to re write distance modulus, log prior, log likelihood and both basic and adaptive mcmc code

# Distance modulus

In [1]:
import numpy as np
from scipy.integrate import quad

def calculate_distance_modulus(z, omega_m, w, h):
  """
  Calculates the dimensionless distance modulus using

  inputs:
  - z: redshift
  - omega_m: density matter parameter
  - w: dark energy equation of state parameter
  - h: dimensionless hubble contant

  outputs: theoretical distance modulus (mu_theoretical)

  """
  c = 299792.458 # speed of light in km/s
  H0 = 100 * h   #Hubble constant in km/s/Mpc

  #Expansion rate
  def expansion_rate(z, omega_m, w):
    return np.sqrt(omega_m * (1+z)**3 +(1-omega_m) * (1+z)**(3*(1+w)))

  #Comoving distance, using scipy integrate, couldnt find a well adopted fitting formula for non flat universe
  def integrand(z, omega_m, w):
    return 1/ expansion_rate(z, omega_m, w)

  result, error = quad(integrand, 0, z, args = (omega_m, w)) #omega and w are fixed for each integration cycle
  d_comoving = (c/H0) * result

  # Transverse co moving distance
  #based on curvature of universe this can be different

  #Flat universe so comoving distance equals transverse co moving
  d_M = d_comoving

  #Luminosty distance
  d_L = (1+z) * d_M

  #Distance modulus
  theoretical_mu = 25 + 5*np.log10(d_L)

  return theoretical_mu

# Likelihood and Prior

In [2]:
#Likelihood function, standard gaussian function

def log_likelihood(mu_obs, mu_model, sigma_mu):

  """
  Computes the logged likelihood for the sample and observed distance modulus

  inputs:
    - mu_obs: observed mu (mu from the data)
    - mu_model: theoretical mu from the model
    - sigma_mu: standard deviation of the observed mu (uncertainty)

  outputs:
    - log likelihood

  """
  return -0.5 * np.sum((mu_obs - mu_model)**2/sigma_mu**2)



# Defining the prior as a function
def log_prior(params):
  """
  Function that sets a uniform prior of the omega parameters and h, based off the planck data

  """
  omega_m, w, h = params

  if 0.1 < omega_m < 0.5 and -2.0 < w < -0.3 and 0.4 < h < 0.9:
    return 0.0 #log of 1 is 0

  else:
    return -np.inf #acceptance probability is 0 if not in the priors upper and lower bounds


# Basic MCMC

In [5]:
# Metropolis Hastings algorithm

def metropolis_hastings(likelihood, z, mu_obs, sigma_mu, n_steps, initial_params, step_size, burn_in, n_walkers):

  """
  Perform basic MCMC to sample from the posterio

  inputs:
    - likelihood: function to compute the likelihood
    - z: redshift
    - mu_obs: observed mu
    - sigma_mu: standard deviation of the observed mu
    - n_steps: Number of steps for MCMC
    - intial_params: initial guesses for [omega_m, w, h] for each walker, has to be array with rows = number of walkers
    - step size: proposal step size for [omega_m, w, h]
    - burn_in : percentage of chain to discard for the burn in period (given as a decimal)
    - n_walkers: number of walkers that are sampling

  outputs:
    Array in shape of (steps after burn in , walker number, 3)
    (3 for the number of parameters in order of omega_m, w, h)

  """
  params = np.array(initial_params) #input as bracket so just make an array
  samples = []
  accepted_samples = np.zeros(n_walkers) #gonna use to calculate acceptance rate

  for step in range (n_steps):
    new_params = np.empty_like(params)   #empty array same size as intial parameters array, need to keep track of new paramters for each walker via matrix

    #Do similar calulation for new parameters as before but just have to loop it for all walkers now
    for i in range(n_walkers):

      #Proposal
      #guess for new parameters with a multivariate gaussian proposal distribution

      #defining the step size for each parameter
      sigma_omega_m = step_size[0]
      sigma_w = step_size[1]
      sigma_h = step_size[2]


      #covariance matrix thing, with off diagonals (correlation between them = 0)
      covariance_matrix = np.diag([sigma_omega_m**2, sigma_w**2, sigma_h**2])

      #Drawing from this new proposal now for each walker/ walker i
      proposed_params = params[i] + np.random.multivariate_normal(np.zeros(3), covariance_matrix)
      omega_m_proposed, w_proposed, h_proposed = proposed_params

      #Priors on omegas and h using the functions we defined earlier
      log_prior_proposed = log_prior(proposed_params)

      #If prior is -inf(when our funciton decides it is not in the bounds of our prior), then reject it
      if np.isneginf(log_prior_proposed):
        new_params[i] = params[i]
        continue

      #PROPOSED PARAMETERS
      #Distance modulus, log likelihood and posterior
      vectorised_distance_mod = np.vectorize(calculate_distance_modulus)
      proposed_mu_model  = vectorised_distance_mod(z, proposed_params[0], proposed_params[1], proposed_params[2])
      proposed_log_likelihood = log_likelihood(mu_obs, proposed_mu_model, sigma_mu)

      #LLog posterior of the proposed params
      proposed_log_posterior = proposed_log_likelihood + log_prior_proposed

      #CURRENT PARAMETERS
      #Distance modulus, log likelihood and posterior of current parameters, from earlier function
      #Distance mod using the parameters at the ith walker, each parameter is a column
      current_mu_model =  vectorised_distance_mod(z, params[i,0], params[i,1], params[i,2])
      current_log_likelihood = log_likelihood(mu_obs, current_mu_model, sigma_mu)

      #Log posterior is the sum of likelihood and prior
      current_log_posterior = current_log_likelihood + log_prior(params[i])



      #Acceptance probalility, based on the change of the current and proposed posteriors
      delta_log_posterior = proposed_log_posterior - current_log_posterior



      #Explicit overflow protection to stop any rogue values messing shit up
      #If the difference in likelihoods is at max python limit, then accept the new proposal with probaiblity 1. Means new parameters are leng
      if delta_log_posterior > 700:
        acceptance_probability = 1.0

      #If difference in likelihood is at min python limit, then dont accept the new proposal at all. Means new parameters are clapped
      elif delta_log_posterior < -700:
        acceptance_probability = 0.0

      #If difference something else then we accept with probability below and randomly sample. Lets us explore parameter space
      else:
        acceptance_probability = min(1, np.exp(delta_log_posterior))


      u = np.random.uniform(0,1) #set the randomness part of accept/ reject

      #Accept proposed move
      if u < acceptance_probability:
        new_params[i] = proposed_params
        accepted_samples[i] += 1
      else:
        new_params[i] = params[i] #chain doesnt move and retrys sampling

    #Updating all the walkers at the same time, outside of the loop, do end of every loop
    params = new_params.copy()
    samples.append(params.copy())

  #Acceptance ratio calculation
  acceptance_ratio = accepted_samples/ n_steps
  print(f"MCMC carried out with {n_steps} steps, and acceptance ratio of each walker {acceptance_ratio}")


  #Number of samples to discard from the chain due to burn in
  burned_chains = int(burn_in * n_steps) #keep integer
  samples_post_burn = samples[burned_chains:] #use everything after the burn in number

  return np.array(samples_post_burn)

# Adaptive MCMC

In [6]:
# Adaptive metropolis hastings algorithm based of Haario etc al 2001

def adaptive_metropolis_hastings(likelihood, z, mu_obs, sigma_mu, n_steps, initial_params, step_size, burn_in, n_walkers, update_interval, target_alpha, learning_rate):
  """
  Perform Metropolis Hastings MCMC to sample from the posterior

  inputs:
    - likelihood: function to compute the likelihood
    - z: redshift
    - mu_obs: observed mu
    - sigma_mu: standard deviation of the observed mu
    - n_steps: Number of steps for MCMC
    - intial_params: initial guesses for [omega_m, w, h] for each walker, has to be array with rows = number of walkers
    - step size: proposal step size for [omega_m, w, h]
    - burn_in : percentage of chain to discard for the burn in period (given as a decimal)
    - n_walkers: number of walkers that are sampling
    - update_interval: number of iterations to update the adaptive parameters
    - target_alpha: target acceptance rate
    - learning_rate: learning rate of the adaptation


  outputs:
    - Array in shape of (steps after burn in , walker number, 3)
      (3 for the number of parameters in order of omega_m, w, h)
    - Alpha all walkers: average acceptance rate of the whole thing
    - History of covariance matricieis
    - History of step sizes

  """

  params = np.array(initial_params) #input as [] so just make an array
  samples = []
  accepted_samples = np.zeros(n_walkers)  #gonna use to calculate acceptance rate

  #New adaptive parameters
  accepted_cycle = np.zeros(n_walkers) #empty array of accepted proposals at each cycle/iteration

  #Proposal
  #guess for new parameters, propsal distribution is a multivariate gaussian distribution now
  #defining the step size for each parameter, step size has to be a 4x1 array
  sigma_omega_m = step_size[0]
  sigma_w = step_size[1]
  sigma_h = step_size[2]


  #Set up covariance matrix
  #Intially set the off diagonals (correlation between the parameters) as 0
  covariance_matrix = np.diag([sigma_omega_m**2, sigma_w, sigma_h**2])

  #add smoothing parameter when updating the covariance matrix later, so that we dont just abruptly jump fully to the new one
  smoothing = 0.2 #keeps 80% of the old covariance, can adjust this later

  #Lists to store the history of the covariance and step size: used to test the pilot runs
  cov_history = []
  step_history = []

  for step in range(n_steps):
    new_params = np.empty_like(params) # empty array same size as initial parameters arry, need to keep track of each walker via matrix


    #Do similar calculation for new parameters as before but just have to loop it for all walkers now
    for i in range(n_walkers):

      #drawing from new proposal now but for each walker/ walker i
      proposed_params = params[i] + np.random.multivariate_normal(np.zeros(3), covariance_matrix)
      omega_m_proposed, w_proposed, h_proposed = proposed_params

      #Priors on the omegas and h using the function for prior
      log_prior_proposed = log_prior(proposed_params)

      #If prior is -inf(when our funciton decides it is not in the bounds of our prior), then reject it
      if np.isneginf(log_prior_proposed):
        new_params[i] = params[i]
        continue

      #PROPOSED PARAMETERS
      #Distance modulus, log likelihood and posterior
      vectorised_distance_mod = np.vectorize(calculate_distance_modulus)
      proposed_mu_model  = vectorised_distance_mod(z, proposed_params[0], proposed_params[1], proposed_params[2])
      proposed_log_likelihood = log_likelihood(mu_obs, proposed_mu_model, sigma_mu)

      #LLog posterior of the proposed params
      proposed_log_posterior = proposed_log_likelihood + log_prior_proposed

      #CURRENT PARAMETERS
      #Distance modulus, log likelihood and posterior of current parameters, from earlier function
      #Distance mod using the parameters at the ith walker, each parameter is a column

      current_mu_model =  vectorised_distance_mod(z, params[i,0], params[i,1], params[i,2])
      current_log_likelihood = log_likelihood(mu_obs, current_mu_model, sigma_mu)

      #Log posterior is the sum of likelihood and prior
      current_log_posterior = current_log_likelihood + log_prior(params[i])



      #Acceptance probalility, based on the change of the current and proposed posteriors
      delta_log_posterior = proposed_log_posterior - current_log_posterior



      #Explicit overflow protection to stop any rogue values messing shit up
      #If the difference in likelihoods is at max python limit, then accept the new proposal with probaiblity 1. Means new parameters are leng
      if delta_log_posterior > 700:
        acceptance_probability = 1.0

      #If difference in likelihood is at min python limit, then dont accept the new proposal at all. Means new parameters are clapped
      elif delta_log_posterior < -700:
        acceptance_probability = 0.0

      #If difference something else then we accept with probability below and randomly sample. Lets us explore parameter space
      else:
        acceptance_probability = min(1, np.exp(delta_log_posterior))




      u = np.random.uniform(0,1) #set the randomness part of accept/ reject

      #Accept proposed move
      if u < acceptance_probability:
        new_params[i] = proposed_params
        accepted_samples[i] += 1
        accepted_cycle[i] += 1 #also add acceptance count for each walker adaptation thing
      else:
        new_params[i] = params[i] #chain doesnt move and retrys sampling

    #Updating all the walkers at the same time, outside of the loop, do end of every loop
    params = new_params.copy()
    samples.append(params.copy())

    #Adaptive update of covariance after every update interval iteration loop
    if (step +1) % update_interval == 0: #check if nth+1 iteration is exact multiple, so saying it has don 100/200/300 steps
      average_acceptance = np.mean(accepted_cycle)/ update_interval #acceptance rate of this cycle (100 steps)

      #Use the last update interval to calculate the variance
      recent_window = samples[-update_interval:]
      recent_samples = np.array(recent_window).reshape(-1,3) #make it 3d

      #Calculate sample covariance and this then includes if there are any correlations between them in the off diagonals
      sample_cov = np.cov(recent_samples, rowvar = False)
      epsilon = 1e-6 #add this to stop any weird 0s messing up calculations
      sample_cov += epsilon *np.eye(4)

      #Scale covariance based on acceptance rate difference
      #Increase covariance matrix if alpha larger than target, decrease if alpha is too small
      #Robbins Munro approximation/ Haario et al.

      scale_factor = np.exp(learning_rate * (average_acceptance - target_alpha))
      new_covariance_matrix = sample_cov * scale_factor


      #update the proposal covariance and then adjust step size using the sqrt of the diagonals of the covariance matrix
      #Add smoothing to the update now
      covariance_matrix = (1-smoothing) * covariance_matrix + smoothing * new_covariance_matrix

      step_size = [np.sqrt(covariance_matrix[0,0]), np.sqrt(covariance_matrix[1,1]), np.sqrt(covariance_matrix[2,2]), np.sqrt(covariance_matrix[3,3])]

      #Saving copies of the covariance and step size to look at when testing the pilot runs
      cov_history.append(covariance_matrix.copy())
      step_history.append(step_size.copy())

      #Reset counter
      accepted_cycle = np.zeros(n_walkers)

  #Acceptance ratio calculation
  acceptance_ratio = accepted_samples/ n_steps
  print(f"MCMC carried out with {n_steps} steps, and acceptance ratio of each walker {acceptance_ratio}")

   #Return avergae acceptance ratio of all the walkers
  alpha_all_walkers = np.mean(acceptance_ratio)
  #Number of samples to discard from the chain due to burn in
  burned_chains = int(burn_in * n_steps) #keep integer
  samples_post_burn = samples[burned_chains:] #use everything after the burn in number

  return np.array(samples_post_burn), alpha_all_walkers, cov_history, step_history


# Gelman Rubin Diagnostic

In [ ]:
def gelman_rubin(chains):
  """
  Function that uses the Gelman-Rubin diagnostic test for convergence, compares variance between multiple chains to the variance within each chain

  input: 2d array of the chain for a particular parameter (rows = value of parameter at iteration, column = walker )

  output: Estimate of R

  """
  #Mean of each chain
  chain_means = np.mean(chains, axis=0) #mean of parameters at each point


  #Overall mean of all the chains across the whole thing
  overall_mean = np.mean(chain_means)

  #Between chain variance
  n, m = chains.shape
  B = (n/(m-1)) * np.sum((chain_means - overall_mean)**2) #lmao this was giving such a weird B value at first because it was summing before squaring, fixed with brackets now

  #Average chain variance
  W = 1/(m) * np.sum(np.var(chains, axis = 0, ddof = 1))

  #Calculate V
  V = ((n-1)/n) * W + ((m+1)/(m*n)) * B

  #Calculate R
  R = np.sqrt(V/W)

  return R

# Autocorrelation

In [ ]:
def autocorrelation(x, lag):

  """
  Calulcated the autocorrelation of array x at a given lag k
  Auto correlation is covariance(X,Y) over standard deviation
  """
  n = len(x)
  if n <= lag:
    return np.nan


  covariance = np.sum((x[:n-lag] - np.mean(x)) * (x[lag:] - np.mean(x)))

  std = np.sum((x - np.mean(x))**2)
  if std == 0:
    return np.nan

  return covariance/std


#Effective sample size

In [ ]:
def eff_sample_size(chain):
  """
  Calculates effective sample size, estimate of sample size that is not related

  inputs: chain of samples from mcmc

  outputs: effective sample size, number
  """
  #Need to do N divided by sum of lag from -inf to inf which simplifies to 1+2* of lag from 1 to T (first odd positive intger for which autocorrelation of that t+1 and t+2 are negative)
  #From chapter 11 in BDA3
  #just gonna use N//2 which is a common implementation for the sum

  N = len(chain)

  rho_sum = 0
  previous_rho = 0
  for i in range(1,N//2):
    rho_t = autocorrelation(chain, i)
    if np.isnan(rho_t):
      continue

    #Stop summing when pt+1 and pt+2 are negative
    if i > 1 and (rho_t + previous_rho) < 0:
      break

    rho_sum += rho_t
    previous_rho = rho_t #to go loop back for the comparison

  ess = N / (1 + 2 * rho_sum)

  return ess

#Well turns out this only does it for one chain rip, but i can use it to calculaurte all the chains ess shown in the next cell

#real ess calculator

def eff_sample_size_multichain(chains):

  """
  Calculated the total effective sample size by using all the chains this time

  Inputs:
  chains: array in the shape of (iterations, n_walker)

  outputs:
  total_ess: a number that says the mean of effective sample size of all the chains
  """
  n_chains = chains.shape[1] #y column of chains array
  ess_values = [eff_sample_size(chains[:,i]) for i in range(n_chains)] #calculate ess for each chain
  total_ess = sum(ess_values) #add up all the chains ess to get one big final ess
  return total_ess